# Description modulating_likelihood.py Model in GPflow

This notebook makes a detailed explanation of the modgp.py module.

In [ ]:
import GPflow
import tensorflow as tf
import numpy as np
import itertools

We first describe the function defined for calculating the weights and evaluation points for several multivariate Hermite-Gauss quadratures. 

In [2]:
def mvhermgauss(means, covs, H, D):
        """
        Return the evaluation locations, and weights for several multivariate Hermite-Gauss quadrature runs.
        :param means: NxD
        :param covs: NxDxD
        :param H: Number of Gauss-Hermite evaluation points.
        :param D: Number of input dimensions. Needs to be known at call-time.
        :return: eval_locations (H**DxNxD), weights (H**D)
        """
        # number of quadrature runs
        N = tf.shape(means)[0]
        
        # get the standard evaluation points and weights
        gh_x, gh_w = GPflow.likelihoods.hermgauss(H)
        
        # build "mesh" to evaluate the D dimensional input function h(X), X \in R^D.
        xn = np.array(list(itertools.product(*(gh_x,) * D)))  # H**DxD
        
        # compute the corresponding weights.
        wn = np.prod(np.array(list(itertools.product(*(gh_w,) * D))), 1)  # H**D
        
        # get the standard deviation
        cholXcov = tf.cholesky(covs)  # NxDxD
        
        # linear transformation of variable to integrate using quadrature.
        X = 2.0 ** 0.5 * tf.batch_matmul(cholXcov, tf.tile(xn[None, :, :], (N, 1, 1)), adj_y=True) + tf.expand_dims(means, 2)  # NxDxH**D
        
        # transpose and reshape the "mesh" 
        Xr = tf.reshape(tf.transpose(X, [2, 0, 1]), (-1, D))  # H**DxNxD
        
        # return transformed evaluation points and scaled weights
        return Xr, wn * np.pi ** (-D * 0.5)

We get the number of quadratures to evaluate from the number of rows of the mean vector. Then we calculate the $H$ weights and evaluation points for a single hermgauss quadrature.

Then, we convert the array gh_x into a tuple by writing "(gh_x,)", then we muliply this tuple by $D$ the number of dimensions of the integral. This product ends up repeating the elements of the tuple $D$ times. This is done to give the input to the itertools.product() function that requires the vectors to make all the possible convinations of its elements (create mesh).
Finally the itertool.product object is converted to list and finally to numpy ndarray.

The procedure to calculate the weights is similar but the "weight mesh" is multiplied through its columns to finally get the corresponding weight for each function evaluation.

Once given the mesh and weights to evaluate the function we need to transform the integration variable in order to get the starndar expression for the Gauss-Hermite quadrature. To do so we need to multiply the integration variables by $\sqrt{2}$ times their corresponding standard deviation plus the corresponding mean.

The standar deviation is computed by the choleski decomposition (the cov matrix is diagonal). This give us N DxD diagonal matrices, or an array of NxDxD.

Once given the evaluation points without any transformation we need to transform them to calculate every of the $N$ quadratures. We first convert the matrix xn to a tensor of dimension $1 \times H^D  \times D$ by using "xn[None, :, :]".  That is why we replicate the tensor xn, i.e. the evaluation points vector (matrix) N times by using "tf.tile(xn[None, :, :], (N, 1, 1))" we multiply each set of evaluation points with the corresponding choleski decomposition "tf.batch_matmul(cholXcov, tf.tile(xn[None, :, :], (N, 1, 1)), adj_y=True)", "adj_y=True" means that the second "tensor" i.e. the repetitions of the evaluation points is transposed.

To the previous value the corresponding means are added. In order to do so the mean tensor should have dimension NxDx1 that does the expression "tf.expand_dims(means, 2)", expands the tensor means to have dimension one in dimension index 2, then we get dimension(means) = NxDx1

The obtained tensor is transposed $H^D \times N \times D$.

the final tensor is reshaped to have dimension $-1 \times D$ (the evaluation points for each dimension $D$). 

Finally the function returns the transformed evaluation points and the weights scaled by $\frac{1}{\pi^{-D/2}}$.


Now we describe the likelihood Class used for the modulated GP.

In [ ]:
class ModLik(GPflow.likelihoods.Likelihood):
    def __init__(self):
        GPflow.likelihoods.Likelihood.__init__(self)
        self.noise_var = GPflow.param.Param(1.0)

    def logp(self, F, Y):
        f, g = F[:, 0], F[:, 1]
        y = Y[:, 0]
        sigma_g = 1./(1 + tf.exp(-g))  # squash g to be positive
        mean = f * sigma_g
        return GPflow.densities.gaussian(y, mean, self.noise_var).reshape(-1, 1)

    def variational_expectations(self, Fmu, Fvar, Y):
        # calculate evaluation points and weights
        Xr, w = mvhermgauss(Fmu, tf.matrix_diag(Fvar), 10, 2)
        
        # reshape the weights
        w = tf.reshape(w, [-1, 1])
        
        # get the evaluation points
        f, g = Xr[:, 0], Xr[:, 1]
        
        # replicate the data H**D times
        y = tf.tile(Y, [100, 1])[:, 0]
        
         # squash g to be positive
        sigma_g = 1./(1 + tf.exp(-g)) 
        
        # calculate mean
        mean = f * sigma_g
        
        # evaluate the likelihood
        evaluations = GPflow.densities.gaussian(y, mean, self.noise_var)
        
        # transpose the evaluations
        evaluations = tf.transpose(tf.reshape(evaluations, tf.pack([tf.size(w), tf.shape(Fmu)[0]])))
        
        #return the weighted evaluations
        return tf.matmul(evaluations, w)

The ModLik likelihood class heritages from the parent Class "GPflow.likelihoods.Likelihood". We define the following methods:

#### Constructor:
The constructor of the parten Class is called first. Then the variance noise is defined

#### logp:
This method calculates and returns the value of the likelihood of each observation in the vector Y. The inputs are the latent functions, that is the matrix F, and the observed vector Y.

#### variational_expectations:
In this method the $N$ expectations are calculated. First, given the mean vectors and variance vectors for the variational distributions over f and g, the evaluation points and the corresponding weights are calculated. The variance vector is transform into a diagonal matrix using "tf.matrix_diag(Fvar)", 10 evaluation points per dimension and 2 dimensions, that is $10 \times 10 = 100$ evaluation points.

The weights are reshaped in a column vector, and the evaluation points are extracted.

Then the observed data is replicated by the total number of evaluation points by dimension, that is $H^D$, and we get "y" with all the elements organized in one single list. This is necesary to evaluate our log_lik function at each quadrature evaluation point.

After that the function evaluations are reshaped and transposed (again) and finally they are multiplied by the corresponding weight.


